# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [18]:
from selenium import webdriver as wd # cria o navegador
from selenium.webdriver.common.by import By # localizar itens de um site
from selenium.webdriver.common.keys import Keys # permite clicar teclas no teclado


navegador = wd.Chrome()

# Passo 1: Entrar no google
navegador.get('https://www.google.com/')
    
# Passo 2: Pesquisar a cotação do dolar
navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('Cotação dolar')

navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

# Passo 3: Pegar a cotação do dolar
cotacao_dolar = navegador.find_element(By.XPATH, '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
      
print(cotacao_dolar)

# Passo 4: Pegar a cotação do euro
navegador.get('https://www.google.com/')

navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('Cotação euro')

navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_euro = navegador.find_element(By.XPATH, '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

print(cotacao_euro)

# Passo 5: Pegar a cotação do ouro
navegador.get('https://www.melhorcambio.com/ouro-hoje#:~:text=O%20valor%20do%20grama%20do,em%20R%24%20283%2C56.')

cotacao_ouro = navegador.find_element(By.XPATH, '//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(',' , '.')
print(cotacao_ouro)


navegador.quit()

4.7528
5.095
282.29


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [19]:
# Passo 6: Atualizar a base de dados com as cotações
import pandas as pd

tabela = pd.read_excel('Produtos.xlsx')
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [20]:
# atualizar a cotação de acordo com a moeda correspondente
# as linhas onde a coluna 'Moeda' = 'Dolar

tabela.loc[tabela['Moeda'] == 'Dólar', 'Cotação'] = float(cotacao_dolar)
tabela.loc[tabela['Moeda'] == 'Euro', 'Cotação'] = float(cotacao_euro)
tabela.loc[tabela['Moeda'] == 'Ouro', 'Cotação'] = float(cotacao_ouro)
display(tabela)

# atualizar preço de compra = preço original * cotação
tabela['Preço de Compra'] = tabela['Preço Original'] * tabela['Cotação']

# atualizar o preço de venda = preço de compra * margem
tabela['Preço de Venda'] = tabela['Preço de Compra'] * tabela['Margem']

display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,4.7528,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,5.0950,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,4.7528,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,4.7528,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,5.0950,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,4.7528,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,282.2900,7000.00,1.15,8050.000


,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,4.7528,4752.752472,1.40,6653.853461
1,Carro Renault,4500.00,Euro,5.0950,22927.500000,2.00,45855.000000
2,Notebook Dell,899.99,Dólar,4.7528,4277.472472,1.70,7271.703202
3,IPhone,799.00,Dólar,4.7528,3797.487200,1.70,6455.728240
4,Carro Fiat,3000.00,Euro,5.0950,15285.000000,1.90,29041.500000
5,Celular Xiaomi,480.48,Dólar,4.7528,2283.625344,2.00,4567.250688
6,Joia 20g,20.00,Ouro,282.2900,5645.800000,1.15,6492.670000


### Agora vamos exportar a nova base de preços atualizada

In [21]:
tabela.to_excel('Produtos Novos.xlsx', index=False)
